In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots       
import datetime 
import numpy as np
from scipy.interpolate import UnivariateSpline
from scipy.interpolate import splprep, splev
from collections import OrderedDict
import ipywidgets as widgets
from IPython.display import clear_output, display

# Neuron
from   neuron           import h
import neuron as nrn
nrn.load_mechanisms('mechanisms/single')
from   analysis_functions import *
# import custom functions
from   master_functions import *

#  Setup neuron
h.load_file('stdlib.hoc')
h.load_file('import3d.hoc')

sim = '1100g'

model = 1
cell_type='dspn'
specs = {'dspn': {
                    'N': 71,
                    'lib': 'Libraries/D1_71bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-dMSN_P270-20_1.02_SGA1-m24.swc',
                    },
         'ispn': {
                    'N': 34,
                    'lib': 'Libraries/D2_34bestFit_updRheob.pkl',
                    'model': 2,
                    'morph': 'Morphologies/WT-iMSN_P270-09_1.01_SGA2-m1.swc'}
        }


specs[cell_type]['model'] = model
morphology = specs[cell_type]['morph']

# Get the current working directory
current_wd = os.getcwd()
target = f'model{model}'
# set working directory
wd = os.path.join(current_wd, cell_type, target, 'physiological', 'simulations', f'sim{sim}')
    
os.chdir(wd)
# Get list of files in directory
names = os.listdir()

# Load each pickle file and store in a dictionary
data_dict = {}
filenames = []  # List to store keys (file names without extensions)

for file_name in names:
    if not file_name.endswith('.pickle'):
        continue
    key, _ = os.path.splitext(file_name)
    try:
        data = pd.read_pickle(file_name)
        data_dict[key] = data
        filenames.append(key)  # Don't forget to add the key to filenames if successful
    except Exception as e:
        print(f'Error unpickling {file_name}: {e}')

# unpack to create global variables
for key, value in data_dict.items():
    globals()[key] = value
for key, value in metadata.items():
    globals()[key] = value

os.chdir(current_wd)
d_lambda = 0.05
dend2remove=None
cell, spines, dend_tree = cell_build(cell_type=cell_type, specs=specs, addSpines=True, spine_per_length=spine_per_length, d_lambda=d_lambda, dend2remove=dend2remove)

save_figs = True
show_figs = True

In [ ]:
# spine locations:
dend = metadata['dend_glut']
glutamate_locations = metadata['glutamate_locations']

for sec in cell.dendlist:
    if sec.name() == dend[0]:
        print('start:', np.round(h.distance(sec(glutamate_locations[0])),3), 
              'end:', np.round(h.distance(sec(glutamate_locations[-1])),3))   

In [ ]:
# single egs for fig are from v_dend_tree 0 and 30
simulation = 'Nsim0'
v_dend_tree[simulation]['dists'][0]
v_dend_tree[simulation]['dists'][30]

In [ ]:
# membrane potential plots
bl = 20
max_time = 400
lwd = 0.8

width=600
height=400

df = pd.DataFrame(v_dend_tree[simulation]['v']).transpose()
x = np.arange(0, df.shape[0] * dt, dt)
y = v_dend_tree[simulation]['dists']
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]

traces = [0, 30]
df2plot = df.iloc[:, traces]
y2plot = [y[traces[0]], y[traces[1]]]

color = 'black'
plot_data=[]
for ii in list(range(df2plot.shape[1])):
    plot_data.append(go.Scatter(x=x, y=df2plot.iloc[:, ii], line=dict(color=color, width=lwd), name='{}'.format(round(y2plot[ii], 2))))
# Plot vdata
fig = go.Figure(data=plot_data)
fig.update_layout(
    title='',
    title_x=0.5,
    xaxis_title='time (ms)',
    yaxis_title='membrane potential (mV)',
    xaxis_range = [0,200],
    yaxis_range = [-85, -10],
    legend_title='distance (um)',
    width=width,
    height=height
)

if show_figs:
    fig.show()
if save_figs:
    save_fig(fig=fig, wd=wd, out_name='egs1', ext='svg', width=width, height=height)

In [ ]:
# Cav1.3 plots for single egs
df = extract_mechs(mechs_np2df(i_mechs_dend_tree)[simulation], mech='cal13', scale=1e6)
x = np.arange(0, df.shape[0] * dt, dt)
y = i_mechs_dend_tree[simulation]['dists']
dendrites_dend = i_mechs_dend_tree[simulation]['dendrites']
y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_dend, y)]

idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]

# traces to plot
traces = [0, 30]
df2plot = df.iloc[:, traces]
y2plot = [y[traces[0]], y[traces[1]]]

color = 'black'
plot_data=[]
for ii in list(range(df2plot.shape[1])):
    plot_data.append(go.Scatter(x=x, y=df2plot.iloc[:, ii], line=dict(color=color, width=lwd), name='{}'.format(round(y2plot[ii], 2))))
# Plot vdata
fig = go.Figure(data=plot_data)
fig.update_layout(
    title='',
    title_x=0.5,
    xaxis_title='time (ms)',
    yaxis_title='current density (pA/cm2)',
    xaxis_range = [0,200],
    yaxis_range = [-250, 5],
    legend_title='distance (um)',    
    width=width,
    height=height
)

if show_figs:
    fig.show()
if save_figs:
    save_fig(fig=fig, wd=wd, out_name='egs2', ext='svg', width=width, height=height)

In [ ]:
cell_coordinates = np.array(mechanisms_3D_distribution['cell_coordinates'])
height=600
width=600
fig_morphology1 = morphology_plot(cell_coordinates, dend_tree, lwd=0.8, color='grey', height=height, width=width)
fig_morphology2 = morphology_plot(cell_coordinates, dend_tree, lwd=0.8, color='grey', s=2, height=height, width=width)

In [ ]:
# directory for saving
save_dir = wd.replace('simulations', 'images')

In [ ]:
s = None # using s to determine smoothness of spline; lower s gives less smoothing
height=600; width=600
    
simulation = 'Nsim0'
X = mechanisms_3D_distribution['distributions']['cal12']
X = [value * 1e4 for value in X] # convert cm/s to um/s
cell_coordinates = mechanisms_3D_distribution['cell_coordinates']
cell_coordinates = np.array(cell_coordinates, dtype=object)

fig3_cal12_pbar = heat2D(cell_coordinates, dend_tree, X, alpha=0.6, lwd=1, zmin=0, zmax=0.1, 
    title='Ca<sub>v</sub>1.2', scale_title = '\u03BCm/s', height=height, width=width, s=s)

X = mechanisms_3D_distribution['distributions']['cal13']
X = [value * 1e4 for value in X] # convert cm/s to um/s

fig3_cal13_pbar = heat2D(cell_coordinates, dend_tree, X, alpha=0.6, lwd=1, zmin=0, zmax=0.01, 
    title='Ca<sub>v</sub>1.3', scale_title = '\u03BCm/s', height=height, width=width, s=s)


X = mechanisms_3D_distribution['distributions']['cav32']
X = [value * 1e4 for value in X] # convert cm/s to um/s

fig3_cav32_pbar = heat2D(cell_coordinates, dend_tree, X, alpha=0.6, lwd=1, zmin=0, zmax=0.1, 
    title='Ca<sub>v</sub>3.2', scale_title = '\u03BCm/s', height=height, width=width, s=s)

X = mechanisms_3D_distribution['distributions']['cav33']
X = [value * 1e4 for value in X] # convert cm/s to um/s
fig3_cav33_pbar = heat2D(cell_coordinates, dend_tree, X, alpha=0.6, lwd=1, zmin=0, zmax=0.01, 
    title='Ca<sub>v</sub>3.3', scale_title = '\u03BCm/s', height=height, width=width, s=s)

In [ ]:
# morphology + heatmaps of channel distributions; do not save 
figures0 = {
    'morphology': fig_morphology2,
    'fig3_cal12_pbar': fig3_cal12_pbar,
    'fig3_cal13_pbar': fig3_cal13_pbar,
    'fig3_cav32_pbar': fig3_cav32_pbar,
    'fig3_cav33_pbar': fig3_cav33_pbar,    
}

figs2display(figures=figures0, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=False)

In [ ]:
start_time = 150
burn_time = 130

# create graphs of maximum membrane potential, total Ca2+i, Cav1.2, 1.3, 3.2, 3.3 current density
simulation = 'Nsim0'
cell_coordinates = v_all_3D[simulation]['cell_coordinates']
cell_coordinates = np.array(cell_coordinates, dtype=object)
z1, dz1, _ = peaks(zdata=v_all_3D[simulation]['v'], start_time=start_time, burn_time=burn_time, dt=dt)

# absolute V (mV) as here or relative dV
if num_gluts == 1:
    zmax = -20 # -78
if num_gluts == 15:
    zmax = -20 # -45  
if num_gluts == 18:
    zmax = -20  
    
fig1 = heat2D(cell_coordinates, dend_tree, z1, alpha=0.6, lwd=1, zmin=-84, zmax=zmax, 
    title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

# NEURON concentrations output in mM
cell_coordinates = Ca_all_3D[simulation]['cell_coordinates']
cell_coordinates = np.array(cell_coordinates, dtype=object)

z2, dz2, _  = peaks(zdata=Ca_all_3D[simulation]['Ca'], start_time=start_time, burn_time=burn_time, dt=dt)
z2 = [value * 1e3 for value in z2]

if model == 1:
    zmax = 800
else:
    zmax = 200

fig2 = heat2D(cell_coordinates, dend_tree, z2, alpha=0.6, lwd=1, zmin=0, zmax=zmax, 
    title='Ca<sub>i</sub>', scale_title='uM', height=height, width=width, s=s)


# NEURON current output in mA/cm2; scale by 1e6 to pA/cm2
cell_coordinates = i_mechs_3D[simulation]['cell_coordinates']
cell_coordinates = np.array(cell_coordinates, dtype=object)

z3, dz3, _   = peaks(extract_mechs_3D(i_mechs_3D, mech='cal13', simulation=simulation), start_time=start_time, burn_time=burn_time, dt=dt, peak_type='min')
z3 = [value * 1e6 for value in z3]

fig3_cal13 = heat2D(cell_coordinates, dend_tree, z3, alpha=0.6, lwd=1, zmin=-250, zmax=0, 
    colorbar_reverse=True, title='Ca<sub>v</sub>1.3', scale_title='pA/cm<sup>2</sup>', 
    height=height, width=width, s=s)

z4, dz4, _   = peaks(extract_mechs_3D(i_mechs_3D, mech='cal12', simulation=simulation), start_time=start_time, burn_time=burn_time, dt=dt, peak_type='min')
z4 = [value * 1e6 for value in z4]

fig3_cal12 = heat2D(cell_coordinates, dend_tree, z4, alpha=0.6, lwd=1, zmin=-750, zmax=0, 
    colorbar_reverse=True, title='Ca<sub>v</sub>1.2', scale_title='pA/cm<sup>2</sup>', 
    height=height, width=width, s=s)


z5, dz5, _   = peaks(extract_mechs_3D(i_mechs_3D, mech='cav32', simulation=simulation), start_time=start_time, burn_time=burn_time, dt=dt, peak_type='min')
z5 = [value * 1e6 for value in z5]

fig3_cav32 = heat2D(cell_coordinates, dend_tree, z5, alpha=0.6, lwd=1, zmin=-150, zmax=0, 
    colorbar_reverse=True, title='Ca<sub>v</sub>3.2', scale_title='pA/cm<sup>2</sup>', 
    height=height, width=width, s=s)

z6, dz6, _   = peaks(extract_mechs_3D(i_mechs_3D, mech='cav33', simulation=simulation), start_time=start_time, burn_time=burn_time, dt=dt, peak_type='min')
z6 = [value * 1e6 for value in z6]

if model == 1:
    zmin = -25
else:
    zmin = -800
fig3_cav33 = heat2D(cell_coordinates, dend_tree, z5, alpha=0.6, lwd=1, zmin=zmin, zmax=0, 
    colorbar_reverse=True, title='Ca<sub>v</sub>3.3', scale_title='pA/cm<sup>2</sup>', 
    height=height, width=width, s=s)

In [ ]:
# heatmaps
# figures1 = {
#     'fig1': fig1,
#     'fig2': fig2,
#     'fig3_cal12': fig3_cal12,
#     'fig3_cal13': fig3_cal13,
#     'fig3_cav32': fig3_cav32,   
#     'fig3_cav33': fig3_cav33,   
# }

# only require potentials and Cav1.3
figures1 = {
    'fig1': fig1,
    'fig3_cal13': fig3_cal13,
}
figs2display(figures=figures1, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=save_figs)

In [ ]:
# absolute V (mV) as here or relative dV
if num_gluts == 1:
    zmax = -20 # -78
if num_gluts == 15:
    zmax = -20 # -45  
if num_gluts == 18:
    zmax = -20  
 
fig1_dend7 = heat2D(cell_coordinates, dend_tree, z1, dend_name='dend[7]', alpha=0.6, lwd=1, 
    zmin=-84, zmax=zmax, title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

fig1_dend10 = heat2D(cell_coordinates, dend_tree, z1, dend_name='dend[10]', alpha=0.6, lwd=1, 
    zmin=-84, zmax=zmax, title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

fig1_dend12 = heat2D(cell_coordinates, dend_tree, z1, dend_name='dend[12]', alpha=0.6, lwd=1, 
    zmin=-84, zmax=zmax, title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

fig1_dend14 = heat2D(cell_coordinates, dend_tree, z1, dend_name='dend[14]', alpha=0.6, lwd=1, 
    zmin=-84, zmax=zmax, title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

fig1_dend15 = heat2D(cell_coordinates, dend_tree, z1, dend_name='dend[15]', alpha=0.6, lwd=1, 
    zmin=-84, zmax=zmax, title='membrane potential', scale_title = 'mV', height=height, width=width, s=s)

# 
# figures2 = {
#     'fig1_dend7': fig1_dend7,
#     'fig1_dend10': fig1_dend10,
#     'fig1_dend12': fig1_dend12,
#     'fig1_dend14': fig1_dend14,
#     'fig1_dend15': fig1_dend15,   
# }

figures2 = {
    'fig1_dend7': fig1_dend7
}

figs2display(figures=figures2, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=save_figs)

In [ ]:
# to create charge density plots pC/cm2
bl = 20
max_time = 300

df = pd.DataFrame(extract_mechs_3D(i_mechs_3D, mech='cal13', simulation=simulation)).transpose()

x = np.arange(0, df.shape[0] * dt, dt)
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]
ind = int(bl/dt)

areas = [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e3 for col in df.columns] # mA to pA 1e6, ms to secs 1e-3

fig3_cal13_pC = heat2D(cell_coordinates, dend_tree, areas, alpha=0.6, lwd=1, zmin=0, zmax=12, 
       title='Ca<sub>v</sub>1.3', scale_title='pC/cm<sup>2</sup>', height=height, width=width, s=s)

fig3_cal13_pC_dend7 = heat2D(cell_coordinates, dend_tree, areas, dend_name='dend[7]', alpha=0.6, 
    lwd=1, zmin=0, zmax=12, title='Ca<sub>v</sub>1.3', scale_title='pC/cm<sup>2</sup>', height=height, width=width, s=s)

# EPSPs generated at given spine for range of membrane capacitances (3D)
figures3 = {
    'fig3_cal13_pC': fig3_cal13_pC,
    'fig3_cal13_pC_dend7': fig3_cal13_pC_dend7, 
}

figs2display(figures=figures3, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=save_figs)

In [ ]:
remove_text = save_figs

simulation = 'Nsim0'
bl = 20
max_time = 300

if show_figs:
    width=1200
    height=1200
else:
    width=2048
    height=2048

x_range=[0,180]
y_range=[0,300]

# membrane potential
df = pd.DataFrame(v_dend_tree[simulation]['v']).transpose()
x = np.arange(0, df.shape[0] * dt, dt)
y = v_dend_tree[simulation]['dists']
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]
dendrites_v = v_dend_tree[simulation]['dendrites']

# soma dist should be zero
y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_v, y)]

# collect peaks
ind = int(bl/dt)
vpath={
    'peak': [df[col].max() - df[col].iloc[:ind].mean() for col in df.columns],
    'dist': y,
    'dendrites': dendrites_v
}

fig4 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-85, -10], width=width, height=height,
              title='membrane potential', dendrites=dendrites_v, cmin=-85, cmax=-20, remove_text=remove_text)

# major branch 14
df = pd.DataFrame(v_branch[simulation]['dend[14]']['v']).transpose()
x = np.arange(0, df.shape[0] * dt, dt)
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
y = v_branch[simulation]['dend[14]']['dists']
dendrites_v = ['dend[14]'] * len(y)
df = df[idxs]

y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_v, y)]

# collect peaks
vbranch14={
    'peak': [df[col].max() - df[col].iloc[:ind].mean() for col in df.columns],
    'dist': y,
    'dendrites': dendrites_v
}

fig4_branch14 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-85, -10], width=width, height=height,
                     title='membrane potential', dendrites=dendrites_v, cmin=-85, cmax=-20, remove_text=remove_text)

# major branch 12
df = pd.DataFrame(v_branch[simulation]['dend[12]']['v']).transpose()
x = np.arange(0, df.shape[0] * dt, dt)
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
y = v_branch[simulation]['dend[12]']['dists']
dendrites_v = ['dend[12]'] * len(y)
df = df[idxs]

y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_v, y)]

# collect peaks
vbranch12={
    'peak': [df[col].max() - df[col].iloc[:ind].mean() for col in df.columns],
    'dist': y,
    'dendrites': dendrites_v
}

fig4_branch12 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-85, -10], width=width, height=height,
                     title='membrane potential', dendrites=dendrites_v, cmin=-85, cmax=-20, remove_text=remove_text)

# major branch 8 + 10 
df1 = pd.DataFrame(v_branch[simulation]['dend[8]']['v']).transpose()
df2 = pd.DataFrame(v_branch[simulation]['dend[10]']['v']).transpose()
df = pd.concat([df1, df2], axis=1)
# important to rename columns 
df.columns = range(df.shape[1])

x = np.arange(0, df.shape[0] * dt, dt)
idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)

y1 = v_branch[simulation]['dend[8]']['dists'] 
y2 = v_branch[simulation]['dend[10]']['dists']
dendrites_v = ['dend[8]'] * len(y1)+ ['dend[10]'] * len(y2)
y = y1+y2
y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_v, y)]

df = df[idxs]

# collect peaks
vbranch8_10={
    'peak': [df[col].max() - df[col].iloc[:ind].mean() for col in df.columns],
    'dist': y,
    'dendrites': dendrites_v
}

fig4_branch8_10 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-85, -10], width=width, height=height,
                     title='membrane potential', dendrites=dendrites_v, cmin=-85, cmax=-20, remove_text=remove_text)

# membrane potential at non-stimulated spines
df = pd.DataFrame(v_spine_tree[simulation]['v']).transpose()
x = np.arange(0, df.shape[0] * dt, dt)
y = v_spine_tree[simulation]['dists']
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]
dendrites_spine = v_spine_tree[simulation]['dendrites']
y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_spine, y)]

# collect peaks
vspinepath={
    'peak': [df[col].max() - df[col].iloc[:ind].mean() for col in df.columns],
    'dist': y,
    'dendrites': dendrites_spine
}

fig4_spine = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-85, -10], width=width, height=height, 
                    title='membrane potential', dendrites=dendrites_spine, cmin=-85, cmax=-20, remove_text=remove_text)

In [ ]:
# EPSPs generated at given spine for range of membrane capacitances (3D)
figures4 = {
    'fig4': fig4,
    'fig4_branch14': fig4_branch14, 
    'fig4_branch12': fig4_branch12, 
    'fig4_branch8_10': fig4_branch8_10, 
    'fig4_spine': fig4_spine,
}

figs2display(figures=figures4, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=False)

In [ ]:
# # Current density for Ca channels
df_cal12 = extract_mechs(mechs_np2df(i_mechs_dend_tree)[simulation], mech='cal12', scale=1e6)
df_cal13 = extract_mechs(mechs_np2df(i_mechs_dend_tree)[simulation], mech='cal13', scale=1e6)
df_cav32 = extract_mechs(mechs_np2df(i_mechs_dend_tree)[simulation], mech='cav32', scale=1e6)
df_cav33 = extract_mechs(mechs_np2df(i_mechs_dend_tree)[simulation], mech='cav33', scale=1e6)
# df_cal13_spines = extract_mechs(mechs_np2df(i_mechs_spine_tree)[simulation], mech='cal13', scale=1e6)

df = df_cal12
x = np.arange(0, df.shape[0] * dt, dt)
y = i_mechs_dend_tree[simulation]['dists']
dendrites_dend = i_mechs_dend_tree[simulation]['dendrites']
y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_dend, y)]

idxs = (x > (stim_time-bl)) & (x < max_time)
x = np.round(x[idxs] - (stim_time-bl), 10)
df = df[idxs]

# peak in pA/cm2
# charge density pC/cm2 or pA/cm2.s 
ical12={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

fig5_cal12 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-800, 0], width=width, height=height,
                    dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)',
                    title='Ca<sub>v</sub>1.2', colorscale='jet_r', cmin=-750, cmax=0, remove_text=remove_text)

df = df_cal13
df = df[idxs]

ical13={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

fig5_cal13 = plot3D(x, y, df, x_range=x_range, y_range=y_range, z_range=[-250, 0], width=width, height=height,
                    dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)', 
                    title='Ca<sub>v</sub>1.3', colorscale='jet_r', cmin=-250, cmax=0, remove_text=remove_text)

df = df_cav32
df = df[idxs]

icav32={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns], 
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

if model == 1:
    cmin=-200
else:
    cmin=-2000
fig5_cav32 = plot3D(x, y, df, x_range=x_range, y_range=y_range, width=width, height=height,
                    dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)', 
                    title='Ca<sub>v</sub>3.2', colorscale='jet_r', cmin=cmin, cmax=0, remove_text=remove_text)

df = df_cav33
df = df[idxs]

icav33={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

if model == 1:
    cmin=-35
else:
    cmin=-800
fig5_cav33 = plot3D(x, y, df, x_range=x_range, y_range=y_range, width=width, height=height,
                    dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)', 
                    title='Ca<sub>v</sub>3.3', colorscale='jet_r', cmin=cmin, cmax=0, remove_text=remove_text)

In [ ]:
figures5 = {
    'fig5_cal12': fig5_cal12,
    'fig5_cal13': fig5_cal13, 
    'fig5_cav32': fig5_cav32, 
    'fig5_cav33': fig5_cav33,
}

figs2display(figures=figures5, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=False)

In [ ]:
dt = 0.025

x, y, df, dendrites_dend = extract_current_densities(v_branch, i_mechs_3D, simulation,  stim_time, bl, max_time, dend='dend[14]', mech='cal13', dt=dt)

y = [0 if dendrite == 'soma[0]' else value for dendrite, value in zip(dendrites_dend, y)]

ical13_branch14={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

fig5_cal13_branch14 = plot3D(x, y, df, x_range=[0,180], y_range=[0,300], z_range=[-250, 0],
    width=width, height=height, dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)',
    title='Ca<sub>v</sub>1.3', colorscale='jet_r', cmin=-250, cmax=0, remove_text=remove_text)

x, y, df, dendrites_dend = extract_current_densities(v_branch, i_mechs_3D, simulation,stim_time, bl, max_time, dend='dend[12]', mech='cal13', dt=dt)

ical13_branch12={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

fig5_cal13_branch12 = plot3D(x, y, df, x_range=[0,180], y_range=[0,300], z_range=[-250, 0],
    width=width, height=height, dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)',
    title='Ca<sub>v</sub>1.3', colorscale='jet_r', cmin=-250, cmax=0, remove_text=remove_text)

x1, y1, df1, dendrites_dend1 = extract_current_densities(v_branch, i_mechs_3D, simulation,  stim_time, bl, max_time, dend='dend[8]', mech='cal13', dt=dt)
x2, y2, df2, dendrites_dend2 = extract_current_densities(v_branch, i_mechs_3D, simulation,  stim_time, bl, max_time,  dend='dend[10]', mech='cal13', dt=dt)

df = pd.concat([df1, df2], axis=1)
df.columns = range(df.shape[1])
x = x1
y = y1+y2
dendrites_dend = dendrites_dend1 + dendrites_dend2

ical13_branch8_10={
    'peak': [-(df[col].min() - df[col].iloc[:ind].mean()) for col in df.columns],
    'charge density': [-(np.trapz(df[col] - df[col].iloc[:ind].mean(), x)) * 1e-3 for col in df.columns],
    'dist': y,
    'dendrites': dendrites_dend
}

fig5_cal13_branch8_10 = plot3D(x, y, df, x_range=[0,180], y_range=[0,300], z_range=[-250, 0],
    width=width, height=height, dendrites=dendrites_dend, zaxis_title='current density (pA/cm<sup>2</sup>)',
    title='Ca<sub>v</sub>1.3', colorscale='jet_r', cmin=-250, cmax=0, remove_text=remove_text)

In [ ]:
figures6 = {
    'fig5_cal13_branch14': fig5_cal13_branch14,
    'fig5_cal13_branch12': fig5_cal13_branch12, 
    'fig5_cal13_branch8_10': fig5_cal13_branch8_10,
}

figs2display(figures=figures6, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=False)

In [ ]:
# dark grey: #696969; grey: #808080; light grey: #D3D3D3

X = [vbranch8_10['dist'], vbranch12['dist'], vbranch14['dist'], vpath['dist']]
Y = [vbranch8_10['peak'], vbranch12['peak'], vbranch14['peak'], vpath['peak']]
titles = ['', '', '', '']
colors = ['#D3D3D3', '#808080', '#696969', '#CD5C5C']

fig_vpeak = plot_v(X, Y, titles, colors, yname='\u0394V', smooth=2)
fig_vpeak.show()

In [ ]:
X = [ical13_branch8_10['dist'], ical13_branch12['dist'], ical13_branch14['dist'], ical13['dist']]
Y = [ical13_branch8_10['peak'], ical13_branch12['peak'], ical13_branch14['peak'], ical13['peak']]
titles = ['', '', '', '']
colors = ['#D3D3D3', '#808080', '#696969', '#CD5C5C']

fig_current_density = plot_v(X, Y, titles, colors, yrange=[-5, 250], yname='pA/cm\u00B2', smooth=100)
fig_current_density.show()

In [ ]:
X = [ical13_branch8_10['dist'], ical13_branch12['dist'], ical13_branch14['dist'], ical13['dist']]
Y = [ical13_branch8_10['charge density'], ical13_branch12['charge density'], ical13_branch14['charge density'], ical13['charge density']]
titles = ['', '', '', '']
colors = ['#D3D3D3', '#808080', '#696969', '#CD5C5C']

fig_charge_density = plot_v(X, Y, titles, colors, yrange=[-0.1, 12.1], yname='pC/cm\u00B2', smooth=0.2)
fig_charge_density.show()

In [ ]:
height = 1000
width = 1200
figures7 = {
    'fig_vpeak': fig_vpeak,
    'fig_current_density': fig_current_density, 
    'fig_charge_density': fig_charge_density,
}

figs2display(figures=figures7, width=width, height=height, save_dir=save_dir, show_figs=show_figs, save_figs=save_figs)